# Introduction

Measurement of downside risk based on current value of a portfolio or security and expected volatility - expected return is optimal

In addition, a time horizon is needed and a confidence level

VaR allows an investor to make statements such as 99% confidence oevel losses will not exceed $XXX in the next days/weeks/months

VaR has problems such as how large a loss can grow to if volatility moves past the confidence level

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

# format for floats
pd.options.display.float_format = '{:,.2f}'.format

# Set some global variables

In [ ]:
# Condifdence level
CL = 0.95

# Amount to invest
pv = 100000

# Volatility
vol = 0.185

# Time Horizon - 1 month - excluind weekends
t = 21

# Number of trading days in a year
TRADING_DAYS = 252

# Deterministic VaR Calculation

For short time horizons er will be small, and therefore VaR estimations will not be much influenced by it.

VaR is generally not calculated for a year out, 1 year is too far in the future.

Generally users are intereted in the short term, tomorrow, a week.


**For this example**

- 99% or 95% confidence level
- 1 years of trading(252 days)
- Horizon of 1 month (21 days)


In [ ]:
from scipy.stats import norm

cutoff = norm.ppf(CL)
VaR = (pv) * vol * np.sqrt(t/TRADING_DAYS) * cutoff

print("At {:.2f} confidence level, loss will not exceed {:,.2f}".format(CL, VaR))
print("This represents a move of {:.2f} standard deviations below the expected return".format(cutoff))

## Refactor into a Function

In [ ]:
import numpy as np
from scipy.stats import norm

def deterministic_VaR(pv, vol, T, CL):
    
    cutoff = norm.ppf(CL)
    
    return pv * vol * np.sqrt(T) * cutoff

## Try it out

In [ ]:
VaR = deterministic_VaR(pv = pv, vol = vol, T = t/TRADING_DAYS, CL = CL)

display(np.round(VaR,2))

# Monte Carlo VaR Calculation

This is one of quite a few ways to calcualte this, known as a parameterized method.

It uses an underlying distribution that I think the stock market will follow 

Other approaches might use empirical / historical distributions instead.

### Download Prices & Calculate volatility

In [ ]:
import yfinance as yf
from datetime import datetime, timedelta

end_time =  datetime.now()
start_time = end_time - timedelta(days=365)

end = end_time.strftime("%Y-%m-%d")
start = start_time.strftime("%Y-%m-%d")

ticker = 'TSLA'

# Latest Price
prices = yf.download(tickers=ticker, start=start, end=end)[['Adj Close']].copy()

# Log returns
prices['Log Rets'] = np.log(prices['Adj Close'] / prices['Adj Close'].shift(1))

# Dailty std
daily_vol = np.std(prices['Log Rets'])

# Annualized Volatility
vol = daily_vol * TRADING_DAYS ** 0.5

print(f'{ticker} Annualized Volatility: {vol:,.2f}')

### Latest Price and Expected Return

Expected return  - assume 25% but its effect is neglible due to the short term nature of VaR, here its 21 days.

Also caluate the exact value of the portfolio 

In [ ]:
# Latest Price
price = prices['Adj Close'].values[-1:][0]
er = 0.25
qty = np.round(pv / price, 0)
value = qty * price

print(f'{ticker} Price: {price:,.2f}')
print(f'Quantity: {qty}')
print(f'Portfolio Value: {value:,.2f}')


## Compute VaR using Monte Carlo

To directly compare with the deterministic approach above

In [ ]:
def MC_VaR(pv, er, vol, T, iterations):
    
    end = pv * np.exp((er - .5 * vol ** 2) * T + 
                     vol * np.sqrt(T) * np.random.standard_normal(iterations))
    
    ending_values = end - pv
    
    return ending_values

In [ ]:
iterations = 50000

at_risk = MC_VaR(pv=value, er=er, vol=vol, T=t/TRADING_DAYS, iterations=iterations)

at_risk

## Display the distribution

The distribution of possible losses and gains over this month

The red, blue and green verticles are the VaR at confidence levels 99%, 95% and 90%

In [ ]:
import seaborn as sns

sns.set()

ending = at_risk

plt.figure(figsize=(12,8))
chart = sns.distplot(a = at_risk, bins=100, kde=True)

plt.axvline(np.percentile(at_risk, 1), color='r')
plt.axvline(np.percentile(at_risk, 5), color='b')
plt.axvline(np.percentile(at_risk, 10), color='g')

plt.title('Monte Carlo Simulation')
plt.xlabel('Final Value ($M)')
plt.ylabel('Count of Simulations')

plt.savefig('../Output/Monte Carlo 2.png')

## Look at some percentiles

Compare the potential loss calcualted by Monte Carlo vs its deterministic equivalent

In [ ]:
percentiles = [1,5,10]
v99,v95,v90 = np.percentile(at_risk, percentiles) * -1
print(f"At 99% confidence level, loss will not exceed {v99:,.2f}")
print(f"At 95% confidence level, loss will not exceed {v95:,.2f}")
print(f"At 90% confidence level, loss will not exceed {v90:,.2f}")



In [ ]:
v99 = deterministic_VaR(pv=value, vol=vol, T = t/TRADING_DAYS, CL = 0.99)
v95 = deterministic_VaR(pv=value, vol=vol, T = t/TRADING_DAYS, CL = 0.95)
v90 = deterministic_VaR(pv=value, vol=vol, T = t/TRADING_DAYS, CL = 0.90)

print(f"At 99% confidence level, loss will not exceed {v99:,.2f}")
print(f"At 95% confidence level, loss will not exceed {v95:,.2f}")
print(f"At 90% confidence level, loss will not exceed {v90:,.2f}")